# 🧠 Reto 50: Generador de Horarios de Estudio 🧠

## 🏆 Objetivo:

Crear un programa que genere un horario de estudio personalizado en función del tiempo disponible del usuario y las materias que desea estudiar.

## 📝 Requisitos:

1️⃣ Entrada del usuario:  
🔸 Días y horas disponibles para estudiar.  
🔸 Lista de materias o temas a estudiar.  
🔸 Prioridad de cada materia (alta, media, baja).  
2️⃣ Generación del horario:  
🔸 Distribuir el tiempo de estudio en función de la prioridad de cada materia.  
🔸 Evitar sesiones de estudio excesivamente largas (máx. 2 horas seguidas por materia).  
🔸 Incluir descansos cada cierto tiempo (ejemplo: 10 min de descanso por cada 50 min de estudio).  
3️⃣ Salida del programa:  
🔸 Mostrar un horario claro y estructurado.  
🔸 Opcional: Permitir exportar el horario en un archivo .txt o .csv.  

## 📌 Ejemplo de ejecución:

**Entrada:**  
¿Cuántos días a la semana puedes estudiar? 3    
¿Cuántas horas por día? 4  
Ingresa las materias que deseas estudiar (separadas por comas): Matemáticas, Física, Historia  
Asigna prioridad (alta, media, baja) en el mismo orden: alta, media, baja  

**Salida esperada:**  

# 📅 Horario de Estudio Generado
---------------------------------
## 📍 Lunes
- **10:00 - 11:50** -> Matemáticas
- **12:00 - 12:50** -> Física

## 📍 Miércoles
- **10:00 - 11:50** -> Matemáticas
- **12:00 - 12:50** -> Historia

## 📍 Viernes
- **10:00 - 11:50** -> Matemáticas
- **12:00 - 12:50** -> Física
---------------------------------
### Nota:
Se incluyen descansos de **10 min cada 50 min**.

## 🔍 Pistas:

🔹 Usa listas y diccionarios para manejar las materias y sus prioridades.  
🔹 Organiza el tiempo dividiendo las horas disponibles proporcionalmente.  
🔹 Implementa lógica para evitar largas sesiones de estudio sin descansos.  
🔹 Puedes mejorar el programa permitiendo guardar el horario en un archivo `.csv`.  

In [2]:
import csv
import datetime

def validar_entero(mensaje, min_val=1):
    """Solicita un número entero al usuario y lo valida."""
    while True:
        try:
            valor = int(input(mensaje))
            if valor < min_val:
                print(f"⚠️ Ingresa un número mayor o igual a {min_val}.")
                continue
            return valor
        except ValueError:
            print("⚠️ Entrada no válida. Ingresa un número entero.")

def generar_horario(dias, horas_por_dia, materias, prioridades):
    """Genera un horario de estudio basado en las materias y prioridades."""
    tiempo_total = dias * horas_por_dia  # Total de horas disponibles
    prioridad_map = {"alta": 3, "media": 2, "baja": 1}  # Asignar peso a prioridades
    pesos = [prioridad_map.get(p.lower(), 1) for p in prioridades]  # Obtener pesos seguros
    total_pesos = sum(pesos)
    
    tiempo_por_materia = [round(tiempo_total * (peso / total_pesos), 2) for peso in pesos]
    
    horario = []
    hora_inicio = datetime.datetime.strptime("18:00", "%H:%M")  # Comienza a las 18:00 horas
    for dia in range(1, dias + 1):
        dia_horario = {"día": f"Día {dia}", "sesiones": []}
        hora_actual = hora_inicio
        horas_disponibles = horas_por_dia
        
        while horas_disponibles > 0:
            for i, materia in enumerate(materias):
                if tiempo_por_materia[i] <= 0 or horas_disponibles <= 0:
                    continue
                
                sesion = min(2, tiempo_por_materia[i], horas_disponibles)
                inicio = hora_actual.strftime("%H:%M")
                hora_actual += datetime.timedelta(hours=sesion)
                fin = hora_actual.strftime("%H:%M")
                
                dia_horario["sesiones"].append(f"{inicio} - {fin} -> {materia}")
                tiempo_por_materia[i] -= sesion
                horas_disponibles -= sesion
                
                if sesion == 2 and horas_disponibles > 0:
                    hora_actual += datetime.timedelta(minutes=10)  # Descanso de 10 min
        
        horario.append(dia_horario)
    return horario

def mostrar_horario(horario):
    """Muestra el horario generado en formato claro."""
    print("\n📅 Horario de Estudio Generado\n" + "-" * 40)
    for dia in horario:
        print(f"📍 {dia['día']}")
        for sesion in dia["sesiones"]:
            print(f"  - {sesion}")
        print("-" * 40)

def exportar_horario(horario, nombre_archivo="horario_estudio.csv"):
    """Exporta el horario a un archivo CSV."""
    with open(nombre_archivo, mode="w", newline="") as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow(["Día", "Inicio", "Fin", "Materia"])
        for dia in horario:
            for sesion in dia["sesiones"]:
                partes = sesion.split(" -> ")
                horario_str, materia = partes[0], partes[1]
                inicio, fin = horario_str.split(" - ")
                escritor.writerow([dia["día"], inicio, fin, materia])
    print(f"\n📂 Horario exportado exitosamente como '{nombre_archivo}'.")

# Solicitar datos al usuario
dias = validar_entero("¿Cuántos días a la semana puedes estudiar? ")
horas_por_dia = validar_entero("¿Cuántas horas por día? ")

materias = input("Ingresa las materias que deseas estudiar (separadas por comas): ").split(",")
materias = [m.strip() for m in materias]

while True:
    prioridades = input(f"Asigna prioridad (alta, media, baja) en el mismo orden para {materias}: ").split(",")
    prioridades = [p.strip().lower() for p in prioridades]
    if len(prioridades) == len(materias) and all(p in ["alta", "media", "baja"] for p in prioridades):
        break
    print("⚠️ Debes ingresar prioridades válidas para cada materia.")

# Generar y mostrar el horario
horario = generar_horario(dias, horas_por_dia, materias, prioridades)
mostrar_horario(horario)

# Preguntar si desea exportar el archivo
exportar = input("\n¿Deseas exportar el horario en un archivo CSV? (sí/no): ").strip().lower()
if exportar == "sí":
    exportar_horario(horario)


¿Cuántos días a la semana puedes estudiar?  3
¿Cuántas horas por día?  2
Ingresa las materias que deseas estudiar (separadas por comas):  Matématica, Física, Estadística, Programación, Economía
Asigna prioridad (alta, media, baja) en el mismo orden para ['Matématica', 'Física', 'Estadística', 'Programación', 'Economía']:  alta, alta, alta, media, baja



📅 Horario de Estudio Generado
----------------------------------------
📍 Día 1
  - 18:00 - 19:30 -> Matématica
  - 19:30 - 20:00 -> Física
----------------------------------------
📍 Día 2
  - 18:00 - 19:00 -> Física
  - 19:00 - 20:00 -> Estadística
----------------------------------------
📍 Día 3
  - 18:00 - 18:30 -> Estadística
  - 18:30 - 19:30 -> Programación
  - 19:30 - 20:00 -> Economía
----------------------------------------



¿Deseas exportar el horario en un archivo CSV? (sí/no):  no
